In [1]:
from datetime import *
import time
import easyquotation as eq

import pandas as pd

import sys
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common')
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py')
print("sys.path:",sys.path)
import common_pg as common_db

print(eq.__version__)

sys.path: ['/Users/afirez/studio/python/auto-trade/examples/py_quant/datasource/easyquotation', '/usr/local/anaconda3/envs/py38/lib/python38.zip', '/usr/local/anaconda3/envs/py38/lib/python3.8', '/usr/local/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions', '/Users/afirez/.ipython', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py']
pg_host : 192.168.31.110 ,pg_user : postgres ,pg_db : k_house
0.7.5


### 选择行情

In [ ]:
q_sina = eq.use('sina') # 新浪 [sina] # 腾讯 [tencent, qq]

### 获取所有股票行情

In [ ]:
data_map_tatol = q_sina.market_snapshot(prefix=True) # prefix 参数指定返回的行情字典中的股票代码 key 是否带 sz /sh 前缀

data_map_tatol

In [ ]:

def  df_from_map1(map):
    code_list = [key for key in map.keys()]
    code_info_list = [map[key] for key in map.keys()]

    df_list = [pd.DataFrame.from_dict(
    data= item,
    orient='index', 
    # columns= data_item.keys()
    ) for item in code_info_list]

    df_list_t = [df.transpose() for df in df_list]

    df = pd.concat(df_list_t)
    df['code'] = code_list
    return df

df_tatol = df_from_map1(data_map_tatol)

df_tatol[:-5]

In [ ]:
# 0. 准备工作

dbname = "k_house"
#   0.1 创建数据库连接

#  创建数据库 股票分析数据库 的连接 
# conn, curs, engine = common_db.creat_conn_with_pg(dbname=f'{dbname}_for_test')

#  创建数据库 k_house 的连接 
conn2, curs2, engine2 = common_db.creat_conn_with_pg(dbname=dbname)

In [ ]:
table_name = 'stock_zh_a_timekline_test'

df_tatol.to_sql(table_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [ ]:
df = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name}
  """,
   # params = (date,),
  con = engine2,
)

# df.head()
df[:-5]

### 单只股票

In [ ]:
data_map = q_sina.real(['002594']) # 支持直接指定前缀，如 ‘sh000001’

data_map

In [ ]:
code = '002594'
data_item = data_map[code]

df = pd.DataFrame.from_dict(
    data= data_item,
    orient='index', 
    # columns= data_item.keys()
)
df = df.transpose()
df["code"] = [code]
df

### 多只股票

In [ ]:
data_map = q_sina.stocks(['000001','002594'])
data_map

In [ ]:
# code = '002594'
# data_item = data_map[code]


# df = pd.DataFrame.from_dict(
#     data= data_item,
#     orient='index', 
#     # columns= data_item.keys()
# )
# df = df.transpose()
# df["code"] = [code]
# df

def  df_from_map(map):
    code_list = [key for key in map.keys()]
    code_info_list = [map[key] for key in map.keys()]

    df_list = [pd.DataFrame.from_dict(
    data= item,
    orient='index', 
    # columns= data_item.keys()
    ) for item in code_info_list]

    df_list_t = [df.transpose() for df in df_list]

    df = pd.concat(df_list_t)
    df['code'] = code_list
    return df

df = df_from_map(data_map)

df